[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/cozek/OffensEval2020-code/blob/master/notebooks/Eng%20Task%20C%20-%20Ensemble%20DistilRoberta%20AttnMask%20Dropout.ipynb)

# Import Libraries

At the time of our work, we used the following library versions
- numpy 1.18.1
- pandas 1.0.1
- torch 1.2.0
- Cuda 10.0
- python 3.7.0
- sklearn 0.22.1
- tqdm 4.42.1
- nltk 3.4.5

In [31]:
!git clone https://github.com/KennethSunn/Intro-to-NLP-Final-Project-Fall2021.git

Cloning into 'Intro-to-NLP-Final-Project-Fall2021'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 78 (delta 23), reused 37 (delta 3), pack-reused 0
Unpacking objects: 100% (78/78), done.


In [32]:
!git clone https://github.com/huggingface/transformershttps://github.com/KennethSunn/Intro-to-NLP-Final-Project-Fall2021.git
!pip install /content/transformers/

fatal: destination path 'Intro-to-NLP-Final-Project-Fall2021' already exists and is not an empty directory.
Processing ./transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.14.0.dev0-py3-none-any.whl size=3320842 sha256=003c2bfec5977d6e9788de55fa6a09848395f7967afa313770a28f42d7f219e4
  Stored in directory: /tmp/pip-ephem-wheel-cache-q9ccj8hp/wheels/49/62/f4/6730819eed4e6468662b1519bf3bf46419b2335990c77f8767
Successfully built transformers
  Attempting uni

In [33]:
import sys
sys.path.append('/content/OffensEval2020-code/src/')
import collections
from typing import Callable
import numpy as np
np.random.seed(42)
import pandas as pd
from tqdm import notebook
import importlib
import pprint
import nltk
import datetime
import os
from argparse import Namespace

from collections import Counter

In [4]:
import utils.general as general_utils
import utils.transformer.data as transformer_data_utils
import utils.transformer.general as transformer_general_utils
general_utils.set_seed_everywhere()

In [5]:
import logging
logging.basicConfig(level=logging.INFO) 

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
torch.__version__ # we used version 1.2.0


'1.10.0+cu111'

In [7]:
# Import RAdam and Lookahead
from radam.radam import RAdam
from lookahead.optimizer import Lookahead


In [8]:
from transformers import RobertaTokenizer, RobertaModel, RobertaForSequenceClassification, AlbertTokenizer, AlbertModel, AlbertForSequenceClassification

In [9]:
 args = Namespace(
        #use cuda by default
        device = 'cuda' if torch.cuda.is_available() else 'cpu',
    
        #set batch size and number of epochs
        batch_size = 32,
        num_epochs = 20,
    
        #set the learning rate
        learning_rate = 0.0001,

        #location of the train, dev and test csv
        train_val_csv = '/content/OffensEval2020-code/data/eng/task_c_tiny.zip',
        test_csv = '/content/OffensEval2020-code/data/test_data/test_a_tweets.tsv',
    
        #directory to save our models at
        directory = './models/', 
        model_name = 'roberta_attn_trac_task_a.pt',
     
        date = datetime.datetime.now().strftime("%a_%d_%b_%Y/"),
)

## Model save location

In [10]:
directory = args.directory + args.date
if not os.path.exists(directory):
    os.makedirs(directory)
args.directory = directory
print(args.directory)

./models/Mon_13_Dec_2021/


## Load presplit dataset portion
```
Labelled as

IND = 0
GRP = 1
OTH = 2
```

In [11]:
data_df_task_c = pd.read_csv(args.train_val_csv, compression='zip')
print(data_df_task_c.label.value_counts())
print(data_df_task_c.split.value_counts())

0    45769
1     7475
2     3448
Name: label, dtype: int64
train    51165
val       2834
test      2693
Name: split, dtype: int64


In [30]:
data_df_task_c.to_csv('/content/Task_c.csv')

In [28]:
k=data_df_task_c.iloc[51165:53999,:]
test_set.label.value_counts()

0    580
1    190
2     80
Name: label, dtype: int64

In [13]:
test_set=pd.read_csv('test_c.csv')

label_to_int = {'IND': 0, 'GRP':1,'OTH':2}

test_set = test_set.replace({"label": label_to_int})
test_set

,id,text,label,split
0,BC0,@USER Lmao bihhh dis what u do to your homies ...,0,test
1,BC3,"@USER The POTUS is a racist, racist, racist, r...",0,test
2,BC9,"@USER He then grinned, raising his brow. Oh d...",2,test
3,BC10,Niggas priorities be fucked all the way up,1,test
4,BC19,You bitches really be walking around talking s...,1,test
...,...,...,...,...
845,BC1903,Caught that fool Mario Lemos sucking dick behi...,0,test
846,BC1906,"@USER Ag,u a not going to say anything just li...",0,test
847,BC1913,@USER 😂😂she always trynna be on my ass,0,test
848,BC1919,@USER F u all the way to hell. You are absolut...,0,test


## Importing the Roberta Tokeniker and Punkt sentence tokenizer

In [ ]:
class RobertaPreprocessor():
    def __init__(self,transformer_tokenizer,sentence_detector):
        self.transformer_tokenizer = transformer_tokenizer
        self.sentence_detector = sentence_detector
        self.bos_token = transformer_tokenizer.bos_token
        self.sep_token = ' ' + transformer_tokenizer.sep_token + ' '
    def add_special_tokens(self, text):
        sentences = self.sentence_detector.tokenize(text)
        eos_added_text  = self.sep_token.join(sentences) 
        return self.bos_token +' '+ eos_added_text + ' ' + self.transformer_tokenizer.sep_token
!python -c 'import nltk; nltk.download("punkt")'

In [ ]:
roberta_tokenizer =  tokenizer= RobertaTokenizer.from_pretrained('distilroberta-base')
punkt_sentence_detector = nltk.data.load('tokenizers/punkt/english.pickle')

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

In [ ]:
roberta_preproc = RobertaPreprocessor(roberta_tokenizer, punkt_sentence_detector)

In [ ]:
#apply the preprocessor on the exploded dataframe
data_df_task_c['text'] = data_df_task_c['text'].map(roberta_preproc.add_special_tokens)


In [ ]:
#apply the preprocessor on the exploded dataframe
test_set['text'] = test_set['text'].map(roberta_preproc.add_special_tokens)


In [ ]:
class SimpleVectorizer():
    def __init__(self,tokenizer: Callable, max_seq_len: int):
        """
        Args:
            tokenizer (Callable): transformer tokenizer
            max_seq_len (int): Maximum sequence lenght 
        """
        self.tokenizer = tokenizer
        self._max_seq_len = max_seq_len

    def vectorize(self,text :str):
        
        encoded = self.tokenizer.encode_plus(
            text,
            add_special_tokens=False, #already added by preproc
            max_length = self._max_seq_len,
            pad_to_max_length = True,
        )
        ids =  np.array(encoded['input_ids'], dtype=np.int64)
        attn = np.array(encoded['attention_mask'], dtype=np.int64)
        
        return ids, attn

class Vectorizer():
    """Vectorizer with Attention Mask Dropout"""
    def __init__(self,tokenizer: Callable, max_seq_len: int ):
        """
        Args:
            tokenizer (Callable): transformer tokenizer
            max_seq_len (int): Maximum sequence lenght 
        """
        self.tokenizer = tokenizer
        self._max_seq_len = max_seq_len

    def vectorize(self,text :str, mask_prob: float = 0.50, mask_amount:float=0.30):
        """Implements Attention Mask Dropout
        
        Args:
            text (str): The string to vectorize
            mask_prob (float): Probability of the attention mask 
                dropout being applied
            mask_amount (float): Percentage of tokens to mask

        Returns:
            ids (np.array)  : Array to token ids of the text
            attn (np.array) : 0-1 Array of attention masks
        """

        encoded = self.tokenizer.encode_plus(
            text,
            add_special_tokens=False, #already added by preproc
            max_length = self._max_seq_len,
            pad_to_max_length = True,
        )
        ids =  np.array(encoded['input_ids'], dtype=np.int64)
        attn = np.array(encoded['attention_mask'], dtype=np.int64)
        prob = np.random.rand(1)[0]
        if  prob <= mask_prob:
            len_of_sent = np.where(ids==tokenizer.pad_token_id)[0][0]
            amount_to_mask = max(int(len_of_sent * mask_amount ) , 1)
            ids_to_not_attend = [np.random.randint(low=0, high=len_of_sent )
             for i in range(amount_to_mask)]
            attn[ids_to_not_attend]=0
            ids[ids_to_not_attend] = tokenizer.mask_token_id
        return ids, attn

Attention Mask Dropout Example

In [ ]:
v = Vectorizer(roberta_tokenizer, 15) #attention maskdropout vectorizer
sv = SimpleVectorizer(roberta_tokenizer, 15) #simple vectorizer

In [ ]:
sent = "I am alright bro, dont worry about me"
_, attn_masks_dropped = v.vectorize(sent)
attn_masks_dropped

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


array([1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0])

In [ ]:
_, attn_masks = sv.vectorize(sent)
attn_masks

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0])

Importing the ALBERT Tokeniker

In [ ]:
'''
class AlbertPreprocessor():
    def __init__(self,transformer_tokenizer,sentence_detector):
        self.transformer_tokenizer = transformer_tokenizer
        self.sentence_detector = sentence_detector
        self.bos_token = transformer_tokenizer.bos_token
        self.sep_token = ' ' + transformer_tokenizer.sep_token + ' '
    def add_special_tokens(self, text):
        sentences = self.sentence_detector.tokenize(text)
        eos_added_text  = self.sep_token.join(sentences) 
        return self.bos_token +' '+ eos_added_text + ' ' + self.transformer_tokenizer.sep_token

!python -c 'import nltk; nltk.download("punkt")'

# Commented out IPython magic to ensure Python compatibility.
# % pip install sentencepiece

albert_tokenizer =tokenizer= AlbertTokenizer.from_pretrained('albert-base-v1')
punkt_sentence_detector = nltk.data.load('tokenizers/punkt/english.pickle')

albert_preproc = AlbertPreprocessor(albert_tokenizer, punkt_sentence_detector)

#apply the preprocessor on the exploded dataframe
data_df_task_c['text'] = data_df_task_c['text'].map(albert_preproc.add_special_tokens)

#apply the preprocessor on the exploded dataframe
test_set['text'] = test_set['text'].map(albert_preproc.add_special_tokens)
'''

###  Create the dataset class

In [ ]:
class HateDataset(Dataset):
    def __init__(
        self,
        data_df: pd.DataFrame,
        tokenizer: Callable,
        max_seq_length:int = None,
    ):
        """
        Args:
            data_df (pandas.DataFrame): df containing the labels and text
            tokenizer (tokenizer module for the transformer)
        """
        self.data_df = data_df
        self.tokenizer = tokenizer

        if max_seq_length is None:
            self._max_seq_length = self._get_max_len(data_df,tokenizer)
        else:
            self._max_seq_length = max_seq_length

        self.train_df = self.data_df[self.data_df.split == 'train']
        self.train_size = len(self.train_df)

        self.val_df = self.data_df[self.data_df.split == 'val']
        self.val_size = len(self.val_df)

        self.test_df = self.data_df[self.data_df.split == 'test']
        self.test_size = len(self.test_df)
        
        self.simple_vectorize = False,
        self._simple_vectorizer = SimpleVectorizer(tokenizer, self._max_seq_length)
        self._vectorizer = Vectorizer(tokenizer, self._max_seq_length)
        
        self._lookup_dict = {
            'train': (self.train_df, self.train_size),
            'val': (self.val_df, self.val_size),
            'test': (self.test_df, self.test_size)
        }

        self.set_split('train')

        class_counts = data_df.label.value_counts().to_dict()
         #sorted on the basis of class label,eg, 0,1,2..
        cts = sorted([(lbl,cts) for lbl,cts in class_counts.items()], key=lambda x: x[0])
        freq = [ x[1] for x in cts ]
        # print(freq,cts)
        self.class_weights = 1.0/ torch.tensor(freq, dtype=torch.float32)
    
    def flip_simple_vectorizer(self) :
        if self.simple_vectorize:
            self.simple_vectorize=False
        else:
            self.simple_vectorize= True
    
    def _get_max_len(self,data_df: pd.DataFrame, tokenizer: Callable):
        len_func = lambda x: len(self.tokenizer.encode_plus(x)['input_ids'])
        max_len = data_df.text.map(len_func).max() 
        return max_len

    def set_split(self, split="train"):
        """ selects the splits in the dataset using a column in the dataframe """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]
    
    def __len__(self):
        return self._target_size
    
    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets
        
        Args:
            index (int): the index to the data point 
        Returns:
            a dictionary holding the data point's features (x_data) and label (y_target)
        """
        row = self._target_df.iloc[index]
        
        if self._target_split == 'train':
            indices, attention_masks = self._vectorizer.vectorize(row.text)
        else:
            indices, attention_masks = self._simple_vectorizer.vectorize(row.text)

        label = row.label
        return {'x_data': indices,
                'x_attn_mask': attention_masks,
                'x_index': index,
                'y_target': label}
    
    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the dataset
        
        Args:
            batch_size (int)
        Returns:
            number of batches in the dataset
        """
        return len(self) // batch_size

In [ ]:
def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=False, device="cpu", pinned_memory = False, n_workers = 0): 
    """
    A generator function which wraps the PyTorch DataLoader. It will 
      ensure each tensor is on the write device location.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last,
                            pin_memory= pinned_memory,
                            num_workers = n_workers,
                            )
    
    for data_dict in dataloader:
        out_data_dict = {}
        out_data_dict['x_data'] = data_dict['x_data'].to(
            device, non_blocking= (True if pinned_memory else False) 
        )
        out_data_dict['x_attn_mask'] = data_dict['x_attn_mask'].to(
            device, non_blocking= (True if pinned_memory else False) 
        )
        out_data_dict['x_index'] = data_dict['x_index']
        out_data_dict['y_target'] = data_dict['y_target'].to(
            device, non_blocking= (True if pinned_memory else False) 
        )
        yield out_data_dict

In [ ]:
dataset = HateDataset(
    data_df = data_df_task_c,
    tokenizer = albert_tokenizer
)

In [ ]:
test_dataset = HateDataset(
    data_df = test_set,
    tokenizer = albert_tokenizer
)

In [ ]:
assert dataset._max_seq_length <= 512

### Implement Attention Mask Dropout in the DistilRoBERTa   vectorizer

In [ ]:
class SimpleVectorizer():
    def __init__(self,tokenizer: Callable, max_seq_len: int):
        """
        Args:
            tokenizer (Callable): transformer tokenizer
            max_seq_len (int): Maximum sequence lenght 
        """
        self.tokenizer = tokenizer
        self._max_seq_len = max_seq_len

    def vectorize(self,text :str):
        
        encoded = self.tokenizer.encode_plus(
            text,
            add_special_tokens=False, #already added by preproc
            max_length = self._max_seq_len,
            pad_to_max_length = True,
        )
        ids =  np.array(encoded['input_ids'], dtype=np.int64)
        attn = np.array(encoded['attention_mask'], dtype=np.int64)
        
        return ids, attn

class Vectorizer():
    """Vectorizer with Attention Mask Dropout"""
    def __init__(self,tokenizer: Callable, max_seq_len: int ):
        """
        Args:
            tokenizer (Callable): transformer tokenizer
            max_seq_len (int): Maximum sequence lenght 
        """
        self.tokenizer = tokenizer
        self._max_seq_len = max_seq_len

    def vectorize(self,text :str, mask_prob: float = 0.50, mask_amount:float=0.30):
        """Implements Attention Mask Dropout
        
        Args:
            text (str): The string to vectorize
            mask_prob (float): Probability of the attention mask 
                dropout being applied
            mask_amount (float): Percentage of tokens to mask

        Returns:
            ids (np.array)  : Array to token ids of the text
            attn (np.array) : 0-1 Array of attention masks
        """

        encoded = self.tokenizer.encode_plus(
            text,
            add_special_tokens=False, #already added by preproc
            max_length = self._max_seq_len,
            pad_to_max_length = True,
        )
        ids =  np.array(encoded['input_ids'], dtype=np.int64)
        attn = np.array(encoded['attention_mask'], dtype=np.int64)
        prob = np.random.rand(1)[0]
        if  prob <= mask_prob:
            len_of_sent = np.where(ids==tokenizer.pad_token_id)[0][0]
            amount_to_mask = max(int(len_of_sent * mask_amount ) , 1)
            ids_to_not_attend = [np.random.randint(low=0, high=len_of_sent )
             for i in range(amount_to_mask)]
            attn[ids_to_not_attend]=0
            ids[ids_to_not_attend] = tokenizer.mask_token_id
        return ids, attn

Attention Mask Dropout Example

In [ ]:
v = Vectorizer(roberta_tokenizer, 15) #attention maskdropout vectorizer
sv = SimpleVectorizer(roberta_tokenizer, 15) #simple vectorizer

In [ ]:
sent = "I am alright bro, dont worry about me"
_, attn_masks_dropped = v.vectorize(sent)
attn_masks_dropped

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


array([1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0])

In [ ]:
_, attn_masks = sv.vectorize(sent)
attn_masks

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0])

Implement Attention Mask Dropout in the ALBERT vectorizer

In [ ]:
'''
v = Vectorizer(albert_tokenizer, 15) #attention maskdropout vectorizer
sv = SimpleVectorizer(albert_tokenizer, 15) #simple vectorizer

sent = "I am alright bro, dont worry about me"
_, attn_masks_dropped = v.vectorize(sent)
attn_masks_dropped

_, attn_masks = sv.vectorize(sent)
attn_masks

"""###  Create the dataset class"""
'''

###  Create the dataset class

In [ ]:
class HateDataset(Dataset):
    def __init__(
        self,
        data_df: pd.DataFrame,
        tokenizer: Callable,
        max_seq_length:int = None,
    ):
        """
        Args:
            data_df (pandas.DataFrame): df containing the labels and text
            tokenizer (tokenizer module for the transformer)
        """
        self.data_df = data_df
        self.tokenizer = tokenizer

        if max_seq_length is None:
            self._max_seq_length = self._get_max_len(data_df,tokenizer)
        else:
            self._max_seq_length = max_seq_length

        self.train_df = self.data_df[self.data_df.split == 'train']
        self.train_size = len(self.train_df)

        self.val_df = self.data_df[self.data_df.split == 'val']
        self.val_size = len(self.val_df)

        self.test_df = self.data_df[self.data_df.split == 'test']
        self.test_size = len(self.test_df)
        
        self.simple_vectorize = False,
        self._simple_vectorizer = SimpleVectorizer(tokenizer, self._max_seq_length)
        self._vectorizer = Vectorizer(tokenizer, self._max_seq_length)
        
        self._lookup_dict = {
            'train': (self.train_df, self.train_size),
            'val': (self.val_df, self.val_size),
            'test': (self.test_df, self.test_size)
        }

        self.set_split('train')

        class_counts = data_df.label.value_counts().to_dict()
         #sorted on the basis of class label,eg, 0,1,2..
        cts = sorted([(lbl,cts) for lbl,cts in class_counts.items()], key=lambda x: x[0])
        freq = [ x[1] for x in cts ]
        # print(freq,cts)
        self.class_weights = 1.0/ torch.tensor(freq, dtype=torch.float32)
    
    def flip_simple_vectorizer(self) :
        if self.simple_vectorize:
            self.simple_vectorize=False
        else:
            self.simple_vectorize= True
    
    def _get_max_len(self,data_df: pd.DataFrame, tokenizer: Callable):
        len_func = lambda x: len(self.tokenizer.encode_plus(x)['input_ids'])
        max_len = data_df.text.map(len_func).max() 
        return max_len

    def set_split(self, split="train"):
        """ selects the splits in the dataset using a column in the dataframe """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]
    
    def __len__(self):
        return self._target_size
    
    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets
        
        Args:
            index (int): the index to the data point 
        Returns:
            a dictionary holding the data point's features (x_data) and label (y_target)
        """
        row = self._target_df.iloc[index]
        
        if self._target_split == 'train':
            indices, attention_masks = self._vectorizer.vectorize(row.text)
        else:
            indices, attention_masks = self._simple_vectorizer.vectorize(row.text)

        label = row.label
        return {'x_data': indices,
                'x_attn_mask': attention_masks,
                'x_index': index,
                'y_target': label}
    
    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the dataset
        
        Args:
            batch_size (int)
        Returns:
            number of batches in the dataset
        """
        return len(self) // batch_size

In [ ]:
def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=False, device="cpu", pinned_memory = False, n_workers = 0): 
    """
    A generator function which wraps the PyTorch DataLoader. It will 
      ensure each tensor is on the write device location.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last,
                            pin_memory= pinned_memory,
                            num_workers = n_workers,
                            )
    
    for data_dict in dataloader:
        out_data_dict = {}
        out_data_dict['x_data'] = data_dict['x_data'].to(
            device, non_blocking= (True if pinned_memory else False) 
        )
        out_data_dict['x_attn_mask'] = data_dict['x_attn_mask'].to(
            device, non_blocking= (True if pinned_memory else False) 
        )
        out_data_dict['x_index'] = data_dict['x_index']
        out_data_dict['y_target'] = data_dict['y_target'].to(
            device, non_blocking= (True if pinned_memory else False) 
        )
        yield out_data_dict

In [ ]:
dataset = HateDataset(
    data_df = data_df_task_c,
    tokenizer = roberta_tokenizer
)

In [ ]:
test_dataset = HateDataset(
    data_df = test_set,
    tokenizer = roberta_tokenizer
)

In [ ]:
assert dataset._max_seq_length <= 512

# Initialize the Roberta model




In [ ]:
model = RobertaForSequenceClassification.from_pretrained(
    'distilroberta-base',
    num_labels=len(set(data_df_task_c.label)),
)

Downloading:   0%|          | 0.00/316M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.weight'

In [ ]:
model.to(args.device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [ ]:
early_stopping = transformer_general_utils.EarlyStopping(patience=4)

In [ ]:
!nvidia-smi

Sat Dec  4 20:35:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    33W / 250W |   1471MiB / 16280MiB |     17%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
args.num_epochs = 20
args.batch_size = 70

In [ ]:
loss_func = nn.CrossEntropyLoss()

print(f'Using LR:{args.learning_rate}')
base_optimizer = RAdam(model.parameters(), lr = args.learning_rate)
optimizer = Lookahead(optimizer = base_optimizer, k = 5, alpha=0.5 )
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer.optimizer, factor =0.1 ,mode='max',
)

# Initialize the ALBERT model




In [ ]:
'''
model = AlbertForSequenceClassification.from_pretrained(
    'albert-base-v2',
    num_labels=len(set(data_df_task_c.label)),
)

model.to(args.device)

early_stopping = transformer_general_utils.EarlyStopping(patience=4)

!nvidia-smi

args.num_epochs = 20
args.batch_size = 70

loss_func = nn.CrossEntropyLoss()

print(f'Using LR:{args.learning_rate}')
base_optimizer = RAdam(model.parameters(), lr = args.learning_rate)
optimizer = Lookahead(optimizer = base_optimizer, k = 5, alpha=0.5 )
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer.optimizer, factor =0.1 ,mode='max',
)

'''

Downloading:   0%|          | 0.00/316M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.weight'

# Begin Training

In [ ]:
model = AlbertForSequenceClassification.from_pretrained(
    'distilroberta-base',
    num_labels=len(set(data_df_task_c.label)),
)


Downloading:   0%|          | 0.00/316M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.weight'

In [ ]:
model.to(args.device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [ ]:
early_stopping = transformer_general_utils.EarlyStopping(patience=4)

In [ ]:
!nvidia-smi

Sat Dec  4 20:35:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    33W / 250W |   1471MiB / 16280MiB |     17%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
args.num_epochs = 20
args.batch_size = 70

In [ ]:
loss_func = nn.CrossEntropyLoss()

print(f'Using LR:{args.learning_rate}')
base_optimizer = RAdam(model.parameters(), lr = args.learning_rate)
optimizer = Lookahead(optimizer = base_optimizer, k = 5, alpha=0.5 )
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer.optimizer, factor =0.1 ,mode='max',
)

In [ ]:
train_state = general_utils.make_train_state()
train_state.keys()


dict_keys(['train_preds', 'train_indexes', 'train_targets', 'train_accuracies', 'train_f1s', 'train_losses', 'val_preds', 'val_indexes', 'val_targets', 'val_accuracies', 'val_f1s', 'val_losses', 'test_preds', 'test_indexes', 'test_targets', 'test_accuracies', 'test_f1s', 'test_losses', 'batch_preds', 'batch_targets', 'batch_indexes', 'epoch_index'])

In [ ]:
epoch_bar = notebook.tqdm(
    desc = 'training_routine',
    total = args.num_epochs,
    position=0,
    leave = True,
)
dataset.set_split('train')
train_bar = notebook.tqdm(
    desc = 'split=train ',
    total=dataset.get_num_batches(args.batch_size),
    position=0,
    leave=True,
)
dataset.set_split('val')
eval_bar = notebook.tqdm(
    desc = 'split=eval',
    total=dataset.get_num_batches(args.batch_size),
    position=0,
    leave=True,
)

old_val_acc = 0
old_f1 = 0
model_state = None
for epoch_index in range(7):#args.num_epochs
    train_state['epoch_in'] = epoch_index

    dataset.set_split('train')

    batch_generator = generate_batches(
        dataset= dataset, batch_size= args.batch_size, shuffle=True,
        device = args.device, drop_last=False,
        pinned_memory = True, n_workers = 3, 
    )

    running_loss = 0.0
    running_acc = 0.0
    running_f1 = 0.0
    model.train()

    train_bar.reset(
        total=dataset.get_num_batches(args.batch_size),
    )

    for batch_index, batch_dict in enumerate(batch_generator):
        optimizer.zero_grad()
        
        loss,y_pred = model(
            input_ids = batch_dict['x_data'],
            attention_mask =  batch_dict['x_attn_mask'],
            labels= batch_dict['y_target'].unsqueeze(1),
        )[:2]
        
        y_pred = y_pred.view(-1, len(set(dataset.data_df.label)))
                             
#         scheduler.step()
        loss.backward()
        optimizer.step()
                             
        loss_t = loss.item()
        running_loss += (loss_t - running_loss) / (batch_index + 1)
                             
        y_pred = y_pred.detach().cpu()
        batch_dict['y_target'] = batch_dict['y_target'].cpu()
        
        acc_t = transformer_general_utils \
            .compute_accuracy(y_pred, batch_dict['y_target'])
        
        f1_t = transformer_general_utils \
            .compute_macro_f1(y_pred, batch_dict['y_target'])

        train_state['batch_preds'].append(y_pred)
        train_state['batch_targets'].append(batch_dict['y_target'])
        train_state['batch_indexes'].append(batch_dict['x_index'])

        running_acc += (acc_t - running_acc) / (batch_index + 1)
        running_f1 += (f1_t - running_f1) / (batch_index + 1)

        train_bar.set_postfix(loss = running_loss, f1 = running_f1, acc=running_acc,
                             epoch=epoch_index)

        train_bar.update()

    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    train_state['train_accuracies'].append(running_acc)
    train_state['train_losses'].append(running_loss)
    
    train_state['train_preds'].append(
        torch.cat(train_state['batch_preds']).cpu()
    )
    train_state['train_targets'].append(
        torch.cat(train_state['batch_targets']).cpu()
    )
    train_state['train_indexes'].append(
        torch.cat(train_state['batch_indexes']).cpu()
    )
    train_f1 = transformer_general_utils \
                .compute_macro_f1(train_state['train_preds'][-1],
                                  train_state['train_targets'][-1],
                                 )
                                 
    train_state['train_f1s'].append(train_f1)
    
    train_state['batch_preds'] = []
    train_state['batch_targets'] = []
    train_state['batch_indexes'] = []
    
    
    dataset.set_split('val')
    batch_generator = generate_batches(
        dataset= dataset, batch_size= args.batch_size, shuffle=True,
        device = args.device, drop_last=False,
        pinned_memory = True, n_workers = 2, 
    )
    eval_bar.reset(
        total=dataset.get_num_batches(args.batch_size),
    )
    running_loss = 0.0
    running_acc = 0.0
    running_f1 = 0.0
    
    model.eval()
    with torch.no_grad():
        optimizer._backup_and_load_cache()
        for batch_index, batch_dict in enumerate(batch_generator):
            loss, y_pred = model(
                input_ids = batch_dict['x_data'],
                attention_mask =  batch_dict['x_attn_mask'],
                labels= batch_dict['y_target'].unsqueeze(1),
            )[:2]
            y_pred = y_pred.view(-1, len(set(dataset.data_df.label)))
            
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            y_pred = y_pred.detach()
            batch_dict['y_target'] = batch_dict['y_target'].cpu()
            
            acc_t = transformer_general_utils\
                .compute_accuracy(y_pred, batch_dict['y_target'])
            f1_t = transformer_general_utils \
                .compute_macro_f1(y_pred, batch_dict['y_target'])

            train_state['batch_preds'].append(y_pred.cpu())
            train_state['batch_targets'].append(batch_dict['y_target'].cpu())
            train_state['batch_indexes'].append(batch_dict['x_index'].cpu())

            running_acc += (acc_t - running_acc) / (batch_index + 1)
            running_f1 += (f1_t - running_f1) / (batch_index + 1)
            

            eval_bar.set_postfix(loss = running_loss, f1 = running_f1, acc=running_acc,
                                 epoch=epoch_index)
            eval_bar.update()
            
    train_state['val_accuracies'].append(running_acc)
    train_state['val_losses'].append(running_loss)
    
        
    train_state['val_preds'].append(
        torch.cat(train_state['batch_preds']).cpu()
    )

    train_state['val_targets'].append(
        torch.cat(train_state['batch_targets']).cpu()
    )
    train_state['val_indexes'].append(
        torch.cat(train_state['batch_indexes']).cpu()
    )
    val_f1 = transformer_general_utils \
                .compute_macro_f1(train_state['val_preds'][-1],
                                  train_state['val_targets'][-1],
                                 )
                                 
    train_state['val_f1s'].append(val_f1)
    
    train_state['batch_preds'] = []
    train_state['batch_targets'] = []
    train_state['batch_indexes'] = []
    
    torch.save(
        {
            'model':model.state_dict(),
        },
        args.directory + f'_epoc_{epoch_index}_' + args.model_name,
    )
    
    scheduler.step(val_f1)
    early_stopping(val_f1, model)
    optimizer._clear_and_load_backup()
    epoch_bar.set_postfix( best_f1 = early_stopping.best_score, current = val_f1)
    epoch_bar.update()    
    
    if early_stopping.early_stop:
        print("Early stopping")
        break


training_routine:   0%|          | 0/20 [00:00<?, ?it/s]

split=train :   0%|          | 0/730 [00:00<?, ?it/s]

split=eval:   0%|          | 0/40 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/py

EarlyStopping counter: 1 out of 4


...

EarlyStopping counter: 2 out of 4


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/py

EarlyStopping counter: 3 out of 4


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/py

EarlyStopping counter: 4 out of 4
Early stopping


In [ ]:
epoch_index

6

In [ ]:
print(train_state['val_f1s'])

[0.7190917399957661, 0.7345660129162933, 0.7691962208764762, 0.7596123759482151, 0.7529665017277498, 0.7666879945094469, 0.7581459179742979]


In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [ ]:

print('Train:',classification_report(
    y_pred=(torch.argmax(train_state['train_preds'][-1],dim=1) ).cpu().long().numpy(),
    y_true= train_state['train_targets'][-1].cpu().numpy(), 
    digits=4)
)
print('Dev:',classification_report(
    y_pred=(torch.argmax(train_state['val_preds'][-1],dim=1) ).cpu().long().numpy(),
    y_true= train_state['val_targets'][-1].cpu().numpy(), 
    digits=4)
)


Train:               precision    recall  f1-score   support

           0     0.9619    0.9769    0.9693     41307
           1     0.8848    0.8435    0.8636      6746
           2     0.7912    0.7076    0.7471      3112

    accuracy                         0.9429     51165
   macro avg     0.8793    0.8426    0.8600     51165
weighted avg     0.9413    0.9429    0.9419     51165

Dev:               precision    recall  f1-score   support

           0     0.9446    0.9611    0.9528      2288
           1     0.7888    0.7888    0.7888       374
           2     0.6136    0.4709    0.5329       172

    accuracy                         0.9086      2834
   macro avg     0.7823    0.7403    0.7581      2834
weighted avg     0.9039    0.9086    0.9056      2834



In [ ]:
best_run_index = train_state['val_f1s'].index(max(train_state['val_f1s']))
print('Train:',classification_report(
    y_pred=(torch.argmax(train_state['train_preds'][best_run_index],dim=1) ).cpu().long().numpy(),
    y_true= train_state['train_targets'][best_run_index].cpu().numpy(), 
    digits=4)
)
print('Dev:',classification_report(
    y_pred=(torch.argmax(train_state['val_preds'][best_run_index],dim=1) ).cpu().long().numpy(),
    y_true= train_state['val_targets'][best_run_index].cpu().numpy(), 
    digits=4)
)

Train:               precision    recall  f1-score   support

           0     0.9307    0.9659    0.9480     41307
           1     0.7999    0.7425    0.7701      6746
           2     0.6228    0.4075    0.4926      3112

    accuracy                         0.9025     51165
   macro avg     0.7845    0.7053    0.7369     51165
weighted avg     0.8948    0.9025    0.8968     51165

Dev:               precision    recall  f1-score   support

           0     0.9513    0.9567    0.9540      2288
           1     0.7664    0.8422    0.8025       374
           2     0.6639    0.4709    0.5510       172

    accuracy                         0.9121      2834
   macro avg     0.7939    0.7566    0.7692      2834
weighted avg     0.9095    0.9121    0.9096      2834



In [ ]:
def sort_preds(indexes, preds):
    """Sorts the predictions in order, to reverse the effects of shuffle
    done by dataloader"""
    indexes = indexes.cpu().numpy().reshape(-1,1)
    preds = preds.cpu().numpy()
    arr_concat = np.hstack((indexes,preds)) #concat the preds and their indexes
    sort_arr = arr_concat[ arr_concat[:,0].argsort()] #sort based on the indexes
    sorted_preds = np.delete(sort_arr,0,axis=1)
    return sorted_preds

In [ ]:
def get_optimal_models_v2(train_state, split):
    l = zip(train_state[f'{split}_f1s'], range(len(train_state[f'{split}_f1s'])))
    sorted_vals = sorted(l, key = lambda x:x[0], reverse=True)
    model_idxes = [i[1] for i in sorted_vals]
    
    trgts= sort_preds(train_state[f'{split}_indexes'][-1],train_state[f'{split}_targets'][-1].reshape(-1,1))
    total_preds = len(train_state[f'{split}_indexes'])
    init = np.zeros(train_state[f'{split}_preds'][-1].shape)
    max_f1 = 0
    idxes = []
    for i in model_idxes:
        temp = sort_preds(train_state[f'{split}_indexes'][i],train_state[f'{split}_preds'][i])
        temp2 = init+temp
        f1 = f1_score(
            y_pred=temp2.argmax(axis=1),
            y_true= trgts, average ='macro'
        )
        if f1 > max_f1:
            max_f1 = f1
            init = init+temp
            idxes.append(i)
    print(f'Taking preds from {idxes} | Dev f1:{f1}')
    return idxes

In [ ]:
final_optimal_models = get_optimal_models_v2(train_state, 'val')
final_optimal_models

Taking preds from [2, 5] | Dev f1:0.7720562279258867


[2, 5]

# Making preds on the given test set

In [ ]:
test_df = test_set

In [ ]:
def evaluate_testset(model, state, dataset, split,args):
    """Returns the final layer output of our transformer model
    Puts them in the '{split}_*' keys in the state dict
    Args:
        model: A pytorch transformers model
        state: dict to store outputs
        dataset: A pytorch Dataset
        split: The split on which to evaluate the model on
        args: Arguments from namespace, etc
    Returns:
        state: all evaluated output stored in the "test" key
    """
    eval_bar = notebook.tqdm(
        desc = 'evaluation progress: ',
        total=dataset.get_num_batches(args.batch_size),
        position=0,
        leave=False,
    )
    dataset.set_split(split)
    batch_generator = generate_batches(
        dataset= dataset, batch_size= args.batch_size, shuffle=False,
        device = args.device, drop_last=False,
        pinned_memory = True, n_workers = 2, 
    )
    eval_bar.reset(
        total=dataset.get_num_batches(args.batch_size),
    )
    model.eval()
    with torch.no_grad():
        for batch_index, batch_dict in enumerate(batch_generator):
            y_pred = model(
                input_ids = batch_dict['x_data'],
                attention_mask =  batch_dict['x_attn_mask'],
            )[0]
            y_pred = y_pred.view(-1, 3)

            y_pred = y_pred.detach()
            
            state['batch_preds'].append(y_pred.cpu())
            state['batch_indexes'].append(batch_dict['x_index'].cpu())
            
            eval_bar.update()
            
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    state[f'{split}_preds'].append(
        torch.cat(state['batch_preds']).cpu()
    )
    state[f'{split}_indexes'].append(
        torch.cat(state['batch_indexes']).cpu()
    )
    
    state['batch_preds'] = []
    state['batch_indexes'] = []
    
    eval_bar.close()
    return state

In [ ]:
all_model_paths= [os.path.join(args.directory,i) for i in os.listdir(args.directory) if args.model_name in i]
all_model_paths = sorted(all_model_paths, key = lambda x: int(x[31])) #sort by epoch num.
all_model_paths

['./models/Sat_04_Dec_2021/_epoc_0_roberta_attn_trac_task_a.pt',
 './models/Sat_04_Dec_2021/_epoc_1_roberta_attn_trac_task_a.pt',
 './models/Sat_04_Dec_2021/_epoc_2_roberta_attn_trac_task_a.pt',
 './models/Sat_04_Dec_2021/_epoc_3_roberta_attn_trac_task_a.pt',
 './models/Sat_04_Dec_2021/_epoc_4_roberta_attn_trac_task_a.pt',
 './models/Sat_04_Dec_2021/_epoc_5_roberta_attn_trac_task_a.pt',
 './models/Sat_04_Dec_2021/_epoc_6_roberta_attn_trac_task_a.pt']

In [ ]:
chosen_models = [all_model_paths[i] for i in final_optimal_models]

In [ ]:
test_set['text'] = test_set['text'].map(roberta_preproc.add_special_tokens)
test_set

,id,text,label,split
0,BC0,<s> <s> @USER Lmao bihhh dis what u do to your...,0,test
1,BC3,"<s> <s> @USER The POTUS is a racist, racist, r...",0,test
2,BC9,"<s> <s> @USER He then grinned, raising his bro...",2,test
3,BC10,<s> <s> Niggas priorities be fucked all the wa...,1,test
4,BC19,<s> <s> You bitches really be walking around t...,1,test
...,...,...,...,...
845,BC1903,<s> <s> Caught that fool Mario Lemos sucking d...,0,test
846,BC1906,"<s> <s> @USER Ag,u a not going to say anything...",0,test
847,BC1913,<s> <s> @USER 😂😂she always trynna be on my ass...,0,test
848,BC1919,<s> <s> @USER F u all the way to hell. </s> </...,0,test


In [ ]:
test_dataset = HateDataset(
    data_df = test_set,
    tokenizer = roberta_tokenizer
)

In [ ]:
test_state = general_utils.make_train_state()
for model_path in notebook.tqdm(chosen_models, total=len(chosen_models)):
    model.load_state_dict(torch.load(model_path)['model'])
    test_state = evaluate_testset(model, test_state, test_dataset, 'test',args)

  0%|          | 0/2 [00:00<?, ?it/s]

evaluation progress: : 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

evaluation progress:   0%|          | 0/12 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

In [ ]:
test_state['test_preds'][-1].shape

torch.Size([850, 3])

In [ ]:
[test_state['test_preds'][i].size() for i in range(len(test_state['test_preds']))]

[torch.Size([850, 3]), torch.Size([850, 3])]

In [ ]:
torch.zeros_like(test_state['test_preds'][0]).size()

torch.Size([850, 3])

In [ ]:
ensemble_pred = torch.zeros_like(test_state['test_preds'][0])
for i in test_state['test_preds']:
    ensemble_pred += i

In [ ]:
int_to_label = { 0: 'IND', 1:'GRP', 2:'OTH'}

In [ ]:
y_pred = []
t = []
for i in torch.argmax(ensemble_pred, dim=1):
    y_pred.append(i)
    t.append(int_to_label[i.item()])

collections.Counter(t)

Counter({'GRP': 167, 'IND': 652, 'OTH': 31})

In [ ]:
y_true = np.array(test_set.iloc[:,2])
test_acc = accuracy_score(y_pred, y_true)

In [ ]:
test_acc

0.8094117647058824

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_pred, y_true))

              precision    recall  f1-score   support

           0       0.94      0.84      0.89       652
           1       0.63      0.71      0.67       167
           2       0.28      0.71      0.40        31

    accuracy                           0.81       850
   macro avg       0.61      0.75      0.65       850
weighted avg       0.86      0.81      0.83       850



In [ ]:
assert len(t) == len(test_df)

In [ ]:
offeval_task_c_pred_analysis_df = pd.DataFrame(
    data={
        'id':test_df.id,
        'text':test_df.text,
        'label':t,
    }
)

In [ ]:
offeval_task_c_pred_label_df = pd.DataFrame(
    data={
        'id':test_df.id,
        'label':t,
    }
)

In [ ]:
offeval_task_c_pred_analysis_df.to_csv(
    'offeval_task_c_pred_analysis.csv',index=False,
)

In [ ]:
offeval_task_c_pred_label_df.to_csv(
    'offeval_task_c_pred_label.csv', index=False, header=False,
)